In [2]:
'''Documentation



'''

'Documentation\n\n\n\n'

In [3]:
# IMPORT LIBRARIES

In [4]:
import os
import re
import nltk
import pandas as pd
import string

In [5]:
# IMPORT MODULES

In [6]:
os.chdir(r'C:\\Users\\Chris.Cirelli\\Desktop\\Python Programming Docs\\GSU\\Sprint Project\\Docket-Sheet-Classification\\Modules')
import Step1_Module_Ngrams_FreqDist as stp1_Ngrams

In [7]:
# IMPORT DOCKET SHEET WITH PRE-CLASSIFIED TIME PERIODS

In [8]:
Docket_sheet_file = r'C:\Users\Chris.Cirelli\Desktop\Python Programming Docs\GSU\Sprint Project\Docket-Sheet-Classification\Data_Files_applicable_all_code\DocketSheet Classification_70_02.22.2018.xlsx'

In [9]:
# CREATE A DATAFRAME WHOSE INDEX IS THE SET OF WORDS AND COLUMNS THE FREQ DIST FOR EACH STAGE OF THE LIFE CYCLE

In [10]:
def master_code_Pipeline_create_dataframe_setWord_freqDist(Docket_sheet_file, Ngrams = 'Bigrams'):
    '''The purpose of this master code pipeline is to pipeline all of the functions required to create the word frequency distribution dataframe 
       for each life cycle of the Docket Sheet
       
       Input       =     Docket sheet file as an excel spreadsheet.  We will read in using pandas. 
                         'Bigrams', 'Trigrams', 'Quadgrams'
                         
       Operations  = 1.) Import the docket sheet, delimit columns, remove rows where time-period = 0/None
                     2.) Concatenate all of the text located in the text column of the Docket Sheet for which a time period has been assigned.
                         Write the text file to current-working-directory.
                     3.) Import into memor the concatenated text file. 
                     4.) Tokenize text file, clean tokens of punctuation, stop words and limit to isalpha(). 
                     5.) Create a set of from our tokenized and cleaned text.  This set will constitute the basis for our word frequency distribution. 
                     6.) Create a dataframe whose index is the above create set of words.  Generate the word frequency for each life cycle stage for
                         this set of words. Return to the user the dataframe in memory. 
    '''
    
    # 1.) Import Target File / Delimit Columns / Remove Rows if Period = None
    df_Master_DocketSheet_File = stp1_Ngrams.import_docket_sheet_file(Docket_sheet_file)
    
    # 2.) Concatenate Text From Docket Sheet Time Periods
    stp1_Ngrams.create_Concatenated_text_file(df_Master_DocketSheet_File, 'Concatenate_DocketSheet_Text')
    
    # 3.) Import Into Memory Concatenated Text File
    Concat_txt_File = stp1_Ngrams.import_concatTxt('Concatenate_DocketSheet_Text.txt')
    
    # 4.) Clean & Tokenize Concatenated Text File.  Return a List object. 
    Clean_tokenized_concatText = stp1_Ngrams.clean_andTokenize_text(Concat_txt_File)
    
    # Create Ngrams
    Ngram_list = stp1_Ngrams.get_Ngrams(Clean_tokenized_concatText, Ngrams)
        
        
    '''    
    # 5.) Create Set of Ngrams from our Ngram_list.  This will serve as the index for our Freq_Dist dataframe. 
    Set_tokenized_concatText = list(set(Ngram_list))
        
    # 6.) Create & Return Word Frequency Distribution Dataframe
    df_word_freq_final = stp1_Ngrams.create_dataframe_setWord_freqDist(df_Master_DocketSheet_File, 
                                                                       Set_tokenized_concatText, 
                                                                       Ngrams)
    '''
    return Ngram_list


In [11]:
df_Word_Freq_Dist = master_code_Pipeline_create_dataframe_setWord_freqDist(Docket_sheet_file, Ngrams = 'Quadgrams')

Concatenated text created

Concatenated text imported



In [12]:
'''VARIATION OF THE CODE.  
LIMIT THE DATAFRAME BY THE STAGE
CLEAN, TOKENIZE EACH & OBTAIN NGRAMS FOR THAT STAGE

'''

'VARIATION OF THE CODE.  \nLIMIT THE DATAFRAME BY THE STAGE\nCLEAN, TOKENIZE EACH & OBTAIN NGRAMS FOR THAT STAGE\n\n'

In [13]:
df1 = pd.DataFrame({}, index = ['A', 'B', 'D']) 
df1['Test1'] = [1,2,3]

In [14]:
df2 = pd.DataFrame({}, index = ['A', 'E', 'C']) 
df2['Test2'] = [4,5,6]

In [15]:
df1.merge(df2, how = 'outer', left_index = True, right_index = True)

,Test1,Test2
A,1.0,4.0
B,2.0,NaN
C,NaN,6.0
D,3.0,NaN
E,NaN,5.0


In [16]:
def get_Ngram_freq_dist_by_stage(Docket_sheet, Ngram_type):
    
    ### Add an option to chose which Ngram you want.  
    ### Also, an option to chose whether you want an absolute freq or the % of the times the Ngram appears for a given stage. 
    
    df_Master_DocketSheet_File = stp1_Ngrams.import_docket_sheet_file(Docket_sheet)
    Stages = set(df_Master_DocketSheet_File['Time Period'])
    
    # Create a Dataframe to house our Freq Dist 
    df_Freq_Dist = ''
    
    # Dictionary to catch the number of rows per stage. 
    Rows_per_stage = {}
    
    # Iterate over each stage in the Docket_sheet
    for stage in Stages:
        # Limit the DataFrame by Each Stage in succession. 
        delimiter = df_Master_DocketSheet_File['Time Period'] == stage
        df_limited = df_Master_DocketSheet_File[delimiter]
        
        # Dictonary to Capture Ngram Freq by Stage
        Ngram_dictionary = {}
        
        # Count rows to serve as the denominator for our freq dist for each stage. 
        Count_rows = 0
        
        # Identify only the text of each row. 
        for row in df_limited.itertuples():
            
            # Count rows
            Count_rows += 1
            
            # Get Text
            text = row[4]
            
            # Clean & Tokenize the text
            clean_tokenized_text = stp1_Ngrams.clean_andTokenize_text(text)
            
            # Get Ngrams
            Ngrams = stp1_Ngrams.get_Ngrams(clean_tokenized_text, Ngram_type)
            
            # Loop over Ngrams
            for ngram in Ngrams:
                Ngram_dictionary[ngram] = Ngram_dictionary.get(ngram, 0) + 1
        
        # Create an entry in our Rows_per_stage dictonary
        Rows_per_stage[stage] = Count_rows
        
        # Once the Dictionary is fully grown for the stage in question, create a column in our dataframe. 
        
        # If the stage = 1, assign the placeholder object df_Freq_Dist the value of the dataframe created from the
        # Freq Dist of our first Stage. 
        
        if stage < 2:
            df_Freq_Dist = pd.DataFrame(Ngram_dictionary, index = [stage]).transpose()
        # Otherwise, create a new dataframe of each new stage and merge it into our stage 1 dataframe on the index. 
        else:
            df = pd.DataFrame(Ngram_dictionary, index = [stage]).transpose()
            df_Freq_Dist = df_Freq_Dist.merge(df, how = 'outer', left_index = True, right_index = True)
 

    # Once the list of ngrams is complete, return it to the user.
    return df_Freq_Dist
            
    

In [19]:
Test = Test_code(Docket_sheet_file, 'Nograms')